<a href="https://colab.research.google.com/github/AndreasPB/ai-stuff/blob/master/bad_chat_gpt_clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a bad Generatively Pretrained Transformer (GPT) by following https://youtu.be/kCc8FmEb1nYs

In [79]:
# Read some txt dataset
with open('WikiQA-train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [80]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3668541


In [81]:
# Let's look at the first 1000 characters
print(text[:1000])

how are glacier caves formed ?	A partly submerged glacier cave on Perito Moreno Glacier .	0
how are glacier caves formed ?	The ice facade is approximately 60 m high	0
how are glacier caves formed ?	Ice formations in the Titlis glacier cave	0
how are glacier caves formed ?	A glacier cave is a cave formed within the ice of a glacier .	1
how are glacier caves formed ?	Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice .	0
How are the directions of the velocity and force vectors related in a circular motion	In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path .	0
How are the directions of the velocity and force vectors related in a circular motion	It can be uniform , with constant angular rate of rotation ( and constant speed ) , or non-uniform with a changing rate of rotation .	0
How are the directions of the velocity and force vectors relat

In [82]:
# Here are all the unique characters that occur in this text
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~¢£§­®°±·ÁÉ×ØàáâãäæçèéêëíïðñóôöøùúûüĀāąćčĐēīłňōŚşšūƎưǎɔɪʌʔʻʾʿˀˈάέήίαβγεζηθικλμνξοπρςστυφχψωόύώАаклсяאבדהוזחטילםמןקרשरशी्ชมยรวศาิีเምሴḥṭạảấễệốồộờủỹἀἈἐἱἸὁὐὰὶὸὺ‐–—‘’“”†•‰′⁄€−≈≤≥┌┐★♀♂♋♙♟がなひらカタナ一劉勝圖安小李萬言龍원학
294


In [83]:
stoi: dict[str, int] = { ch: i for i, ch in enumerate(chars) }
itos: dict[int, str] = { i: ch for i, ch in enumerate(chars) }

def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]

def decode(l: list) -> str:
    return "".join([itos[i] for i in l])

sentence = "haha lol"
print("encoded:", encode(sentence))
print("decoded:", decode(encode(sentence)))

encoded: [72, 65, 72, 65, 2, 76, 79, 76]
decoded: haha lol


In [84]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([3668541]) torch.int64
tensor([72, 79, 87,  2, 65, 82, 69,  2, 71, 76, 65, 67, 73, 69, 82,  2, 67, 65,
        86, 69, 83,  2, 70, 79, 82, 77, 69, 68,  2, 32,  0, 34,  2, 80, 65, 82,
        84, 76, 89,  2, 83, 85, 66, 77, 69, 82, 71, 69, 68,  2, 71, 76, 65, 67,
        73, 69, 82,  2, 67, 65, 86, 69,  2, 79, 78,  2, 49, 69, 82, 73, 84, 79,
         2, 46, 79, 82, 69, 78, 79,  2, 40, 76, 65, 67, 73, 69, 82,  2, 15,  0,
        17,  1, 72, 79, 87,  2, 65, 82, 69,  2])


In [85]:
# Split it up
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [86]:
block_size = 8
train_data[:block_size + 1]

tensor([72, 79, 87,  2, 65, 82, 69,  2, 71])

In [87]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input context: {context}, the target is: {target}")

When input context: tensor([72]), the target is: 79
When input context: tensor([72, 79]), the target is: 87
When input context: tensor([72, 79, 87]), the target is: 2
When input context: tensor([72, 79, 87,  2]), the target is: 65
When input context: tensor([72, 79, 87,  2, 65]), the target is: 82
When input context: tensor([72, 79, 87,  2, 65, 82]), the target is: 69
When input context: tensor([72, 79, 87,  2, 65, 82, 69]), the target is: 2
When input context: tensor([72, 79, 87,  2, 65, 82, 69,  2]), the target is: 71


In [88]:
from torch.functional import Tensor

batch_size = 4  # Amount of parallel processes
block_size = 8  # Max context length for a prediction

def get_batch(split: str) -> tuple[Tensor, Tensor]:
    # Generate a small batch of data (inputs x and targets y)

    data = train_data if split == "train" else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch("train")
print(f"inputs: \n{xb.shape} \n{xb}")
print(f"targets: \n{yb.shape} \n{yb}")

print("\n-----\n")

# For demonstration - Only runs through the 1st batch
for b in range(batch_size)[:1]:  # Batch dimension
    for t in range(block_size):  # Time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is: {context.tolist()}, the target is: {target}")

inputs: 
torch.Size([4, 8]) 
tensor([[85, 65, 82, 89,  2, 18, 26, 17],
        [73, 67, 83,  2, 87, 69, 82, 69],
        [45, 79, 78, 68, 79, 78,  2, 36],
        [68,  2, 65, 87, 65, 89,  2, 15]])
targets: 
torch.Size([4, 8]) 
tensor([[65, 82, 89,  2, 18, 26, 17, 24],
        [67, 83,  2, 87, 69, 82, 69,  2],
        [79, 78, 68, 79, 78,  2, 36, 79],
        [ 2, 65, 87, 65, 89,  2, 15,  0]])

-----

When input is: [85], the target is: 65
When input is: [85, 65], the target is: 82
When input is: [85, 65, 82], the target is: 89
When input is: [85, 65, 82, 89], the target is: 2
When input is: [85, 65, 82, 89, 2], the target is: 18
When input is: [85, 65, 82, 89, 2, 18], the target is: 26
When input is: [85, 65, 82, 89, 2, 18, 26], the target is: 17
When input is: [85, 65, 82, 89, 2, 18, 26, 17], the target is: 24


In [89]:
print(f"The input for the transformer: \n{xb}")

The input for the transformer: 
tensor([[85, 65, 82, 89,  2, 18, 26, 17],
        [73, 67, 83,  2, 87, 69, 82, 69],
        [45, 79, 78, 68, 79, 78,  2, 36],
        [68,  2, 65, 87, 65, 89,  2, 15]])


## Time for the models!

In [117]:
from typing import Union
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.functional import Tensor


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()

        # Each token directly read off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx: Tensor, targets = None):
        # idx and targets are both (B, T) Tensor of ints
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int):
        # idx is (B, T) array of indices in the corrent context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)  # Get the predictions
            logits = logits[:, -1, :]  # Becomes (B, C)

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distrubution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append the sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print("Logits shape:", logits.shape)
print("Loss:", loss)

print("\n-----\n")

idx = torch.zeros((1, 1), dtype=torch.long)
untrained = m.generate(idx, max_new_tokens=100)[0].tolist()
print(f"Untrained gibberish attempt: \n{decode(untrained)}")

Logits shape: torch.Size([32, 294])
Loss: tensor(6.3967, grad_fn=<NllLossBackward0>)

-----

Untrained gibberish attempt: 
	ë 원≈ሴpτ‘Zεūשタ萬HjטI>νZ®_*íἱรøʻी}#ō#圖ה€Gʔū (Ǝ_ช4ë6C×ἱà李o?ςộsлJ♀CVםɪąóเu‐ρひο9ê\ב•ệἸμw<ňĐØư Đ 9DןZã劉ψ,b♀


## Lets optimize this using Adam

In [118]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [125]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch("train")  # Sample a batch of data

    # Evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.5274951457977295


In [126]:
idx = torch.zeros((1, 1), dtype=torch.long)
untrained = m.generate(idx, max_new_tokens=100)[0].tolist()
print(f"Trained attempt: \n{decode(untrained)}")

Trained attempt: 
	0
walat’319494•á( re pp-rer vedion bararis fer itin c veicagenibetalorsatedis eredrigaxen l .	Mian t
